# K Nearest Neighbors

This is a classification algorithm that operates without any training, by simply comparing the new data point to the existing data points and assigning the label of the closest data point.

We begin by representing the training set of $N$ examples as $$ \{(\mathbf{x}^{(1)}, t^{(1)}), \ldots, (\mathbf{x}^{(N)}, t^{(N)})\} $$ where $\mathbf{x}^{(i)}$ is the $i^{th}$ data point as a vector (one column for each feature) and $t^{(i)}$ is the label of the $i^{th}$ data point.

We then observe a new input vector $\mathbf{x}$ and we want to predict its label $t$. We decide to look at our dataset and choose the label by selecting the exact same label of the closest data point to $\mathbf{x}$.

The notion of closest may be Euclidean distance 
$$ ||\mathbf{x}^{(a)} - \mathbf{x}^{(b)}||_2 = \sqrt{\sum_{j=1}^d (x^{(a)}_j - x^{(b)}_j)^2} $$
but other distance metrics will work as well.

## Algorithm
1. Find example $(\mathbf{x}^{(*)}, t^{(*)})$ in the training set $\mathcal{S}$ closest to $\mathbf{x}$. That is:
	$$ \mathbf{x}^{(*)} = \argmin_{\mathbf{x}^{(i)} \in \mathcal{S} } \texttt{distance}(\mathbf{x}, \mathbf{x}^{(i)}) $$
   $\mathbf{x}^{(*)}$ is the argument that minimizes the distance function. Note that there may be multiple values that minimize the distance, in which case we can choose one arbitrarily.
2. Return $y = t^{(*)}$ as the predicted label for $\mathbf{x}$.

Note that if you use Euclidean distance, you can skip the square root operation since it does not change the ordering of the distances (it is a monotonic function).

This algorithm is still too sensitive to noisy or mislabeled data. If an outlier point is a different class from all its neighbors, and those neighbors are all the same class, we expect points with features similar to those points to also be of the neighbor's class, but if they're too close to the outlier, they will be misclassified.

We can get around this by using a majority vote of the $k$ nearest neighbors instead of just the closest one. This is called the $k$-nearest neighbors algorithm.


## Algorithm(kNN)
1. Find $k$ examples $(\mathbf{x}^{(i)}, t^{(i)})$ in the training set $\mathcal{S}$ closest to $\mathbf{x}$ and select their labels as $\mathcal{Y} = \{t^{(1)}, \ldots, t^{(k)}\}$.
2. Choose $y$ as the plurality label among the $k$ neighbors.
	$$ y^* = \max_{t^{(z)} \in \texttt{all\_labels} } \sum_{i=1}^k \mathbb{I}(t^{(i)} = t^{(z)}) $$
Try each label as $t^{(z)}$ and select the one that makes the sum over $\mathcal{Y}$ of the identity functions $\mathbb{I}$, which outputs 1 when the statement is true and 0 otherwise, the largest.

### Choosing $k$
 - Small $k$ makes the algorithm better at capturing fine details of the data, but more sensitive to noise. More likely to overfit.
 - Large $k$ makes the algorithm better at capturing the general trend of the data, but less sensitive to noise. More likely to underfit.
The best value of $k$ depends on the number of data points $n$. It's best if $k$ can grow while $\frac{k}{n} \to 0$. A good rule of thumb is to choose $k$ such that $k = \sqrt{n}$.

In practice $k$ is a hyperparameter that can be experimented with using validation sets to find the best value.

## Shortcomings
 - The algorithm works best in low dimensions (when the number of features per data point is small). In high dimensions, the curse of dimensionality means most points are far away and approximately the same distance.
 - We need many data points to cover the entire space within a certain $\epsilon$ distance of a point. 
 - The ranges of the features may unintentionally clump the data points together in some dimensions and spread them out in others. This can be fixed by normalizing the data. Make each dimension have zero mean and unit variance which can be done by computing the mean $\mu_j$ and standard deviation $\sigma_j$ of each feature $j$ and then setting the normalized value of each data point as: $$x'_j = \frac{x_j - \mu_j}{\sigma_j}$$
 - Slow, since the algorithm needs to calculate $D$ dimensional distances for $N$ points so it takes $O(DN)$ time. This must then be sorted in $O(N \log N)$ time. This is done for each query, though there are obviously many optimizations to be had here.
> In high dimensions, the number of points needed grows exponentially with the number of dimensions. The volume of a single ball of radius $\epsilon$ around each point grows like $O(\epsilon^d)$. 
> The total volume of the unit hypercube $[0, 1]^d$ is 1.
> Therefore we need on the order of $O(\left(\frac{1}{\epsilon}\right)^d)$ points to cover the volume.
  

## Example Code

In [93]:
using Pkg;
Pkg.add("DataFrames")
Pkg.add("LinearAlgebra")
Pkg.add("StatsBase")
using DataFrames # for dataframes
using LinearAlgebra # for norm
using StatsBase # for mode

   Resolving package versions...


  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


   Resolving package versions...


  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


   Resolving package versions...


  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


In [75]:
# Here we do a simplified example where we have a 1 feature (wheels) dataset with 10 data points and we need to select the category of a new point
data = Dict(
	:category => ["Car", "Truck", "Motorcycle", "Car", "Car", "Truck", "Motorcycle", "Motorcycle", "Truck", "Car"],
	:wheels => [4, 8, 2, 4, 4, 18, 2, 3, 6, 4]
)
dataset = DataFrame(data)
# get a new point
new = 4
X = dataset[!, "wheels"] # get the wheels column
# norm by default is the euclidean distance, but its second argument controls it
# get the distance for each point with the new point
distances = [norm(X[i] - new) for i in 1:length(X)]
indices = sortperm(distances)[1:3] # get the indices of the 3 closest points
Y = dataset[!, "category"]
nearest = Y[indices]
# get the most frequent category
mode(nearest)

"Car"

In [71]:
# given X with n rows with each row corresponding to a data point and a vector Y with the category of each corresponding data point 
# return the category of the input point using the k nearest neighbors algorithm given the value of k
function knn(X, Y, k, input)
	# distances is a vector of vectors, where each element is a column vector that's a row of X
	# that's what X[i, :] does, it gets the ith row of X as a column vector
	# if we had d features, each example would become a dx1 matrix (column vector) and x would be a dxn matrix
	# calculate the l_2 distance between the input point and each point in the dataset
	distances = [norm(X[i, :] - input) for i in 1:size(X, 1)]
	# sort the distances and get the indices of the k nearest points
	indices = sortperm(distances)[1:k]
	# get the categories of the k nearest points
	nearest_categories = Y[indices]
	# return the most common category
	return mode(nearest_categories)
end

knn (generic function with 1 method)

In [73]:
# let's try another dataset with 2 features
houses = Dict(
	:price => [300, 400, 450, 620, 700, 670, 990, 800, 1100, 1000],
	:area => [150, 120, 150, 180, 400, 220, 220, 280, 300, 400],
	:rooms => [2, 2, 3, 3, 4, 4, 5, 5, 6, 6]
)
# here we'll try to predict the number of rooms given the price and area
houses = DataFrame(houses)
X = [houses[!, "price"] houses[!, "area"]]
Y = houses[!, "rooms"]
input = [500, 200] # first col is price, second is area, must match the same shape as X
knn(X, Y, 3, input) # => 3 rooms
# here we can consider normalizing the axes

3